In [50]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import sklearn 
import string
import re
from IPython.display import display, Latex, Markdown

In [29]:
df = pd.read_csv('data/best_value_data.csv')
df.head()

,INSTNM,ST_FIPS,STABBR,LATITUDE,LONGITUDE,SAT_AVG,UNITID,OPEID6,CONTROL,MAIN,...,COUNT,DEBTMEDIAN,DEBTPAYMENT10YR,DEBTMEAN,TITLEIVCOUNT,EARNINGSCOUNT,MD_EARN_WNE,IPEDSCOUNT1,IPEDSCOUNT2,BESTVALUE
0,Alabama A & M University,1,AL,34.783368,-86.568502,929.0,100654.0,1002,Public,1,...,25,36250,376,37194,33,33,54300,17.0,16.0,1.497931
1,Alabama A & M University,1,AL,34.783368,-86.568502,929.0,100654.0,1002,Public,1,...,46,31505,327,33085,32,31,25400,24.0,33.0,0.806221
2,Alabama A & M University,1,AL,34.783368,-86.568502,929.0,100654.0,1002,Public,1,...,22,34809,361,36482,33,31,54500,19.0,11.0,1.565687
3,Alabama A & M University,1,AL,34.783368,-86.568502,929.0,100654.0,1002,Public,1,...,PrivacySuppressed,36928,383,38900,24,24,45800,13.0,17.0,1.240251
4,Alabama A & M University,1,AL,34.783368,-86.568502,929.0,100654.0,1002,Public,1,...,39,29956,311,30970,30,30,24000,28.0,28.0,0.801175


In [34]:
df_data = df[['INSTNM', 'CIPDESC', 'DEBTMEAN','MD_EARN_WNE','BESTVALUE' ]]
df_data.head()

,INSTNM,CIPDESC,DEBTMEAN,MD_EARN_WNE,BESTVALUE
0,Alabama A & M University,"Computer and Information Sciences, General.",37194,54300,1.497931
1,Alabama A & M University,Teacher Education and Professional Development...,33085,25400,0.806221
2,Alabama A & M University,"Electrical, Electronics and Communications Eng...",36482,54500,1.565687
3,Alabama A & M University,Mechanical Engineering.,38900,45800,1.240251
4,Alabama A & M University,"Liberal Arts and Sciences, General Studies and...",30970,24000,0.801175


In [44]:
#taking average of best value score based on major
df_major = df_data
columns = ['Major','Average Best Value Score']
unique_major = df_data.CIPDESC.unique()
tempdf =pd.DataFrame(columns = columns)
for major in unique_major:
    group_of_major = df_major.loc[df_major['CIPDESC']== major]
    tempdf = tempdf.append({'Major' : major , 'Average Best Value Score' : group_of_major['BESTVALUE'].mean()} , ignore_index=True)
    
df_major = tempdf
df_major.head()


,Major,Average Best Value Score
0,"Computer and Information Sciences, General.",2.670863
1,Teacher Education and Professional Development...,1.448755
2,"Electrical, Electronics and Communications Eng...",2.998553
3,Mechanical Engineering.,2.856958
4,"Liberal Arts and Sciences, General Studies and...",1.357815


In [45]:
#taking average of best value score based on school
df_school = df_data
columns = ['School','Average Best Value Score']
unique_school = df_data.INSTNM.unique()
tempdf =pd.DataFrame(columns = columns)
for school in unique_school:
    group_of_school = df_school.loc[df_school['INSTNM']== school]
    tempdf = tempdf.append({'School' : school , 'Average Best Value Score' : group_of_school['BESTVALUE'].mean()} , ignore_index=True)
    
df_school = tempdf
df_school.head()

,School,Average Best Value Score
0,Alabama A & M University,1.095372
1,University of Alabama at Birmingham,1.570209
2,University of Alabama in Huntsville,2.004191
3,Alabama State University,0.776373
4,The University of Alabama,1.635502


In [48]:
#Adding column for if best value score is below this threshold than its classified as a bad investment
threshold = 1.5 
temp = []
for entry in df_major['Average Best Value Score']:
    if(entry > 1.5):
        temp.append(True)
    else:
        temp.append(False)

df_major['Good Investment'] = temp
df_major.head()

,Major,Average Best Value Score,Good Investment
0,"Computer and Information Sciences, General.",2.670863,True
1,Teacher Education and Professional Development...,1.448755,False
2,"Electrical, Electronics and Communications Eng...",2.998553,True
3,Mechanical Engineering.,2.856958,True
4,"Liberal Arts and Sciences, General Studies and...",1.357815,False


In [49]:
threshold = 1.5 
temp = []
for entry in df_school['Average Best Value Score']:
    if(entry > 1.5):
        temp.append(True)
    else:
        temp.append(False)

df_school['Good Investment'] = temp
df_school.head()

,School,Average Best Value Score,Good Investment
0,Alabama A & M University,1.095372,False
1,University of Alabama at Birmingham,1.570209,True
2,University of Alabama in Huntsville,2.004191,True
3,Alabama State University,0.776373,False
4,The University of Alabama,1.635502,True
